# Modelo de Jansen y Rit en Red

Ahora que ya hemos visto el funcionamiento de una columna cortical, conectaremos una red de ellas, a través de sus funciones de input $p(t)$. 


La matriz de conectividad estructural que utilizaremos proviene de https://doi.org/10.1523/JNEUROSCI.5068-13.2014 

## 1) Importar librerías 

In [ ]:
# -*- coding: utf-8 -*-

import BOLDModel as BD ##función hemodinámica
import JansenRitModel as JR ##script modelo

import numpy as np
from scipy import signal
import time
import matplotlib.pyplot as plt
import psutil,os
try:
    psutil.Process(os.getpid()).nice(psutil.HIGH_PRIORITY_CLASS) ##para darle prioridad en procesador (más rápido)
except Exception:
    pass




## 2) Parámetros de simulación

In [ ]:

#Simulation parameters
JR.dt = 1E-3 #Integration step
JR.teq = 60 #Simulation time for stabilizing the system
JR.tmax = 200 + JR.teq * 2 #Length of simulated signals
ttotal = JR.teq + JR.tmax #Total simulation time
JR.downsamp = 10 #Downsampling to reduce the number of points        
Neq = int(JR.teq / JR.dt / JR.downsamp) #Number of points to discard
Nmax = int(JR.tmax / JR.dt / JR.downsamp) #Number of points of simulated signals
Ntotal = Neq + Nmax #Total number of points of simulation

seed = 0 #Random Seed

#Network parameters
JR.alpha = 1.3#Long-range pyramidal to pyramidal coupling

##########################NOTA: PODEMOS CAMBIAR LA MATRIZ
SCmat = np.loadtxt('data/structural_Deco_AAL.txt') ##matriz original por DTI de Deco 2014
# SCmat = np.loadtxt("data/SC_opti_25julio.txt") #matriz con aumento "artificial" de la diagonal para compensar por infraestimación interhemisférica

norm = np.mean(np.sum(SCmat,0)) #Normalization factor, lo necesitamos para que el modelo no se salga de al bifurcación
M = SCmat/norm
JR.M = SCmat

JR.nnodes = len(JR.M) #number of nodes

#Node parameters
JR.r0 = 0.56 #Slope of pyramidal neurons sigmoid function
JR.p = 4.8 #Input mean
JR.sigma = 1 * np.sqrt(JR.dt) #Input standard deviation
JR.C4 = (0.3 + JR.alpha * 0.3 / 0.5) * 135 # Feedback inhibition

JR.seed = seed

##echar un ojo a la matriz de conectividad
plt.figure(1)
plt.clf()
plt.title("SC matrix")
plt.imshow(SCmat,cmap="jet")
plt.colorbar()
plt.show()


## 3) Simulamos y guardamos cuánto tarda en correr

In [ ]:
init1 = time.time()
y, time_vector = JR.Sim(verbose = True)
pyrm = JR.C2 * y[:,1] - JR.C4 * y[:,2] + JR.C * JR.alpha * y[:,3] #EEG-like output of the model
end1 = time.time()
print([end1 - init1])



## 4) ploteamos EEG

In [ ]:


#Plot EEG-like signals
plt.figure(1)
plt.clf()
plt.suptitle("EEG stuff")

plt.subplot(211)
plt.title("EEG-like signals")
plt.plot(time_vector[Neq:(Neq + 10000)], pyrm[Neq:(Neq + 10000),0])
plt.tight_layout()
print(np.mean(np.corrcoef(pyrm[Neq:,:].T)))


#Power Spectral Density

#Welch method to stimate power spectal density (PSD)
#Remember: dt = original integration step, dws = downsampling           
window_length = 20 #in seconds
PSD_window = int(window_length / JR.dt / JR.downsamp) #Welch time window
PSD = signal.welch(pyrm[Neq:,:] - np.mean(pyrm[Neq:,:], axis = 0), fs = 1 / JR.dt / JR.downsamp, 
                    nperseg = PSD_window, noverlap = PSD_window // 2, 
                    scaling = 'density', axis = 0)
freq_vector = PSD[0] #Frequency values
PSD_curves =  PSD[1] #PSD curves for each node

#Power spectral density functions
plt.subplot(223)
plt.title("spectral power")
plt.plot(freq_vector[1:-2], np.mean(10 * np.log10 (PSD_curves[1:-2,:]),axis=1))
plt.ylabel("power density (log10)")
plt.xlabel("frequency")
plt.tight_layout()
plt.show()



## 5) ploteamos BOLD y matriz de conectividad funcional BOLD

In [ ]:
    
#%%
#fMRI-BOLD response
init3 = time.time()
rE = JR.s(pyrm, JR.r0)  #Firing rates!!



BOLD_signals = BD.Sim(rE, JR.nnodes, JR.dt * JR.downsamp)
BOLD_signals = BOLD_signals[Neq:,:]

BOLD_downsamp = 100
BOLD_dt = JR.dt * JR.downsamp * BOLD_downsamp
BOLD_signals = BOLD_signals[::BOLD_downsamp,:]

#Filter the BOLD-like signal between 0.01 and 0.1 Hz
Fmin, Fmax = 0.01, 0.1
a0, b0 = signal.bessel(3, [2 * BOLD_dt * Fmin, 2 * BOLD_dt * Fmax], btype = 'bandpass')
BOLDfilt = signal.filtfilt(a0, b0, BOLD_signals[:,:], axis = 0)
cut0, cut1 = Neq // BOLD_downsamp, (Nmax - Neq) // BOLD_downsamp
BOLDfilt = BOLDfilt[cut0:cut1,:]
    
#Static Functional Connectivity (sFC) Matrix

end3 = time.time()

print(end3 - init3)



In [ ]:

###simulated functional connectivity matrix
sFC = np.corrcoef(BOLDfilt.T)

#load empirical matrices
empFC_W = np.loadtxt("data/mean_mat_W_8dic24.txt")
empFC_N3 = np.loadtxt("data/mean_mat_N3_8dic24.txt")

similarity_W = np.corrcoef(sFC.flatten(),empFC_W.flatten())[0,1]
similarity_N3 = np.corrcoef(sFC.flatten(),empFC_N3.flatten())[0,1]
print(f"sim-emp correlation (W)= {similarity_W:.3f}")
print(f"sim-emp correlation (N3)= {similarity_N3:.3f}")

#Filtered BOLD-like signals
plt.figure(5)
plt.clf()
plt.subplot(221)
plt.plot(BOLDfilt)
plt.tight_layout()

#sFC matrix
plt.subplot(222)
plt.title("simulated matrix")
plt.imshow(sFC, cmap = 'jet', vmin = 0, vmax = 1)
plt.colorbar()
plt.tight_layout()


#sFC matrix
plt.subplot(223)
plt.title("empirical matrix (W)")
plt.imshow(empFC_W, cmap = 'jet', vmin = 0, vmax = 1)
plt.colorbar()

plt.subplot(224)
plt.title("empirical matrix (N3)")
plt.imshow(empFC_N3, cmap = 'jet', vmin = 0, vmax = 1)
plt.colorbar()


plt.tight_layout()
plt.show()

###qué observamos?



## Ejercicio!

Cargando la matriz de conectividad funcional de W y N3, busca el parámetro óptimo de coupling ($\alpha$) que ajusta cada estado. Puedes utilizar cualquier función de similaridad entre matrices.

1) El parámetro óptimo es mayor o menor para N3? Compara las matrices simuladas y empíricas.
2) Cómo cambia el parámetro óptimo usando distintas funciones de similaridad?
3) Qué ocurre si cambiamos la matriz de conectividad?

In [ ]:

########HINT 1) Crear una funcion que tome un parametro y entregue la matriz de conectividad funcional
########HINT 2) Crear una funcion que tome un parametro y entregue la matriz de conectividad funcional
# alfas = np.linspace(0.5,1.5,10)

# for a,alfa in enumerate(alfas):
#     JR.alpha = 1.3#Long-range pyramidal to pyramidal coupling
    